# Operations with Time Series of Different Frequencies

Economic time series are often of annual, quarterly, monthly, daily, or some other more specialized frequency. Some are completely irregular; for example, earnings revisions for a stock may arrive at any time. The two main tools for frequency conversion and realignment are the resample and reindex methods. resample converts data to a fixed frequency while reindex conforms data to a new index. Both support optional interpolation (such as forward filling) logic.

Let’s consider a small weekly time series:

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [2]:
ts1 = Series(np.random.randn(3),
            index = pd.date_range('13-06-2022', periods = 3, freq = 'W-Wed'))

C:\Users\mbkmu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3378: UserWarning: Parsing '13-06-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
ts1

2022-06-15   -0.474621
2022-06-22   -0.078615
2022-06-29    0.540962
Freq: W-WED, dtype: float64

If you resample this to business daily (Monday-Friday) frequency, you get holes on the days where there is no data:

In [4]:
ts1.resample('B').sum()

2022-06-15   -0.474621
2022-06-16    0.000000
2022-06-17    0.000000
2022-06-20    0.000000
2022-06-21    0.000000
2022-06-22   -0.078615
2022-06-23    0.000000
2022-06-24    0.000000
2022-06-27    0.000000
2022-06-28    0.000000
2022-06-29    0.540962
Freq: B, dtype: float64

In [5]:
for i,j in ts1.resample('B'):
    print(DataFrame([i,j]))

                                                   0
0                                2022-06-15 00:00:00
1  2022-06-15   -0.474621
Freq: W-WED, dtype: flo...
                                         0
0                      2022-06-16 00:00:00
1  Series([], Freq: W-WED, dtype: float64)
                                         0
0                      2022-06-17 00:00:00
1  Series([], Freq: W-WED, dtype: float64)
                                         0
0                      2022-06-20 00:00:00
1  Series([], Freq: W-WED, dtype: float64)
                                         0
0                      2022-06-21 00:00:00
1  Series([], Freq: W-WED, dtype: float64)
                                                   0
0                                2022-06-22 00:00:00
1  2022-06-22   -0.078615
Freq: W-WED, dtype: flo...
                                         0
0                      2022-06-23 00:00:00
1  Series([], Freq: W-WED, dtype: float64)
                                     

Of course, using 'ffill' as the fill_method forward fills values in those gaps. This is a common practice with lower frequency data as you compute a time series of values on each timestamp having the latest valid or “as of” value:

In [6]:
ts1.resample('B').agg('ffill')

2022-06-15   -0.474621
2022-06-16   -0.474621
2022-06-17   -0.474621
2022-06-20   -0.474621
2022-06-21   -0.474621
2022-06-22   -0.078615
2022-06-23   -0.078615
2022-06-24   -0.078615
2022-06-27   -0.078615
2022-06-28   -0.078615
2022-06-29    0.540962
Freq: B, dtype: float64

In practice, upsampling lower frequency data to a higher, regular frequency is a fine solution, but in the more general irregular time series case it may be a poor fit. Consider an irregularly sampled time series from the same general time period:

In [7]:
dates = pd.DatetimeIndex(['2012-6-12', '2012-6-17', '2012-6-18', '2012-6-21', '2012-6-22', '2012-6-29'])

In [8]:
ts2 = Series(np.random.randn(6), index = dates)

In [9]:
ts2

2012-06-12    0.819515
2012-06-17   -0.333380
2012-06-18   -0.061451
2012-06-21   -1.013532
2012-06-22   -0.466960
2012-06-29   -0.113835
dtype: float64

If you wanted to add the “as of” values in ts1 (forward filling) to ts2. One option would be to resample both to a regular frequency then add, but if you want to maintain the date index in ts2, using reindex is a more precise solution:

In [13]:
ts1.reindex(ts2.index, method = 'ffill')

2012-06-12   NaN
2012-06-17   NaN
2012-06-18   NaN
2012-06-21   NaN
2012-06-22   NaN
2012-06-29   NaN
dtype: float64

In [14]:
ts2 + ts1.reindex(ts2.index, method='ffill')

2012-06-12   NaN
2012-06-17   NaN
2012-06-18   NaN
2012-06-21   NaN
2012-06-22   NaN
2012-06-29   NaN
dtype: float64

### Using periods instead of timestamps

Periods (representing time spans) provide an alternate means of working with different frequency time series, especially financial or economic series with annual or quarterly frequency having a particular reporting convention. For example, a company might announce its quarterly earnings with fiscal year ending in June, thus having Q-JUN frequency. Consider a pair of macroeconomic time series related to GDP and inflation:

In [15]:
gdp = Series([1.78, 1.94, 2.08, 2.01, 2.15, 2.31, 2.46],
            index=pd.period_range('1984Q2', periods=7, freq='Q-SEP'))

In [17]:
infl = Series([0.025, 0.045, 0.037, 0.04],
            index=pd.period_range('1982', periods=4, freq='A-DEC'))

In [20]:
print(gdp,'\n\n\n', infl)

1984Q2    1.78
1984Q3    1.94
1984Q4    2.08
1985Q1    2.01
1985Q2    2.15
1985Q3    2.31
1985Q4    2.46
Freq: Q-SEP, dtype: float64 


 1982    0.025
1983    0.045
1984    0.037
1985    0.040
Freq: A-DEC, dtype: float64


Unlike time series with timestamps, operations between different-frequency time series
indexed by periods are not possible without explicit conversions. In this case, if we
know that infl values were observed at the end of each year, we can then convert to *Q-SEP* to get the right periods in that frequency:

In [21]:
infl_q = infl.asfreq('Q-Sep', how = 'end')

In [22]:
infl_q

1983Q1    0.025
1984Q1    0.045
1985Q1    0.037
1986Q1    0.040
Freq: Q-SEP, dtype: float64

That time series can then be reindexed with forward-filling to match gdp:

In [24]:
infl_q.reindex(gdp.index)

1984Q2      NaN
1984Q3      NaN
1984Q4      NaN
1985Q1    0.037
1985Q2      NaN
1985Q3      NaN
1985Q4      NaN
Freq: Q-SEP, dtype: float64

In [25]:
infl_q.reindex(gdp.index, method='ffill')

1984Q2    0.045
1984Q3    0.045
1984Q4    0.045
1985Q1    0.037
1985Q2    0.037
1985Q3    0.037
1985Q4    0.037
Freq: Q-SEP, dtype: float64